Generate plot of SFNO-generated vs E3SMv2 distribution of precipitation.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import xarray as xr

from dask.diagnostics import ProgressBar
from matplotlib import pyplot as plt

from fme_diags.histogram import compute_histograms, plot_time_mean_histogram
from fme_diags.utils import load_config, open_autoregressive_inference

_ = xr.set_options(keep_attrs=True)

In [2]:
CONFIG_KEY = "topo-lr3.00e-04"
PREDS_START_DATE = "0031-01-01"
FLIP_LAT = False
CHUNKS = {"timestep": 178}

In [3]:
config_url = load_config("./preds.yaml")[CONFIG_KEY]["url"]
ds = open_autoregressive_inference(config_url, start=PREDS_START_DATE, flip_lat=FLIP_LAT, chunks=CHUNKS)
ds

,Array,Chunk
Bytes,7.05 GiB,88.00 MiB
Shape,"(2, 1, 14597, 180, 360)","(2, 1, 178, 180, 360)"
Count,167 Tasks,83 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,7.05 GiB,88.00 MiB
Shape,"(2, 1, 14597, 180, 360)","(2, 1, 178, 180, 360)"
Count,167 Tasks,83 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [4]:
da = ds["surface_precipitation_rate"]

with ProgressBar():
    da = da.resample(time="1D").mean("time").compute()

[########################################] | 100% Completed |  6.0s


In [5]:
hist_range = da.min().compute().item(), da.max().compute().item()

No latitude weighting:

In [ ]:
%%time

precip_hist_ds = compute_histograms(da, n_bins=300, hist_range=hist_range, lat_weighted=False)
precip_hist_ds

In [ ]:
fig, ax = plot_time_mean_histogram(precip_hist_ds, log=True, density=True, histtype="step")
fig.show()

With latitude weighting:

In [ ]:
%%time

lat_weighted_precip_hist_ds = compute_histograms(da, n_bins=300, hist_range=hist_range, lat_weighted=True)

In [ ]:
fig, ax = plot_time_mean_histogram(lat_weighted_precip_hist_ds, log=True, density=True, histtype="step")
fig.show()

Tropics, no latitude weighting:

In [ ]:
da_tropics = da.sel(lat=slice(15, -15))
hist_range_tropics = da_tropics.min().compute().item(), da_tropics.max().compute().item()

In [ ]:
%%time

precip_hist_ds_tropics = compute_histograms(da_tropics, n_bins=300, hist_range=hist_range_tropics, lat_weighted=False)
precip_hist_ds_tropics

In [ ]:
fig, ax = plot_time_mean_histogram(precip_hist_ds_tropics, log=True, density=True, histtype="step")
fig.show()